In [1]:
import hashlib
from pymerkle import *

In [2]:
class MerkleTree:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None
        self.hash = None

    def compute_hash(self):
        data_hash = hashlib.sha256(self.data.encode()).hexdigest().encode()
        self.hash = data_hash
        return data_hash

    def insert(self, data_list):
        if len(data_list) == 0:
            return None

        mid = len(data_list) // 2
        self.data = data_list[mid]

        if len(data_list) > 1:
            self.left = MerkleTree(None)
            self.left.insert(data_list[:mid])
            self.right = MerkleTree(None)
            self.right.insert(data_list[mid + 1:])

        self.compute_hash()

    def print_tree(self, level=0, branch=None):
        if self.data is None:
            return

        prefix = " " * level * 4
        if branch == "left":
            prefix += "L-> "
        elif branch == "right":
            prefix += "R-> "

        print(prefix + f"- {self.data}")
        if self.left:
            self.left.print_tree(level + 1, "left")
        if self.right:
            self.right.print_tree(level + 1, "right")

    def search(self, target):
        if self.data == target:
            return True
        if self.left and target < self.data:
            return self.left.search(target)
        elif self.right and target > self.data:
            return self.right.search(target)
        return False

    def get_proof_of_inclusion(self, target):
        proof = []
        node = self
        while node.data != target:
            if target < node.data and node.left:
                proof.append((node.data, node.right.hash))
                node = node.left
            elif target > node.data and node.right:
                proof.append((node.data, node.left.hash))
                node = node.right
            else:
                raise ValueError("Target not found in the Merkle tree")
        return proof

In [3]:

data_list = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
merkle_tree = MerkleTree(None)
merkle_tree.insert(data_list)

print("Merkle Tree:")
merkle_tree.print_tree()

target_data = 'A'
proof = merkle_tree.get_proof_of_inclusion(target_data)
print("\nProof of Inclusion for", target_data, ":", proof)

Merkle Tree:
- F
    L-> - C
        L-> - B
            L-> - A
        R-> - E
            L-> - D
    R-> - I
        L-> - H
            L-> - G
        R-> - J

Proof of Inclusion for A : [('F', b'a83dd0ccbffe39d071cc317ddf6e97f5c6b1c87af91919271f9fa140b0508c6c'), ('C', b'a9f51566bd6705f7ea6ad54bb9deb449f795582d6529a0e22207b8981233ec58'), ('B', None)]
